In [1]:
import jax.numpy as jnp
from jax import config, random
import matplotlib.pyplot as plt

config.update("jax_enable_x64", True)
%config InlineBackend.figure_format='retina'

In [2]:
! git init .
! git remote add origin https://github.com/VLSF/SDC
! git pull origin main

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 107 (delta 52), reused 83 (delta 31), pack-reused 0
Receiving objects: 100% (107/107), 22.30 KiB | 7.43 MiB/s, done.
Resolving deltas: 100% (52/52), done.
From https://github.com/VLSF/SDC
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [3]:
from integrators import RK4, Explicit_Euler, Implicit_Euler
from misc import Chebyshev, utils, equations

The goals are:

1. Provide examples how the code can be used
2. Check the order of the methods

# Test equations

1. Van der Pol oscillator
  \begin{equation}
    \begin{split}
      \dot{y}_1(t) &= y_2(t),\\
      \dot{y}_2(t) &= \frac{1}{\epsilon}(1 - y_1(t)^2 y_2(t)) - y_1(t).\\
    \end{split}
  \end{equation}
  Nonlinear system stiff for small $\epsilon$. No exact solution available.

2. Logistic equation
  \begin{equation}
      \begin{split}
          \dot{y}(t) &= y(t)(1 - y(t)),\\
          y(0) &= \frac{1}{2}.
      \end{split}
  \end{equation}
  Non-stiff nonlinear scalar equations with exact solution $y(t) = 1 \big / (1 + \exp(-t))$.

3. Harmonic oscillator
  \begin{equation}
      \begin{split}
          \dot{y}_1(t) &= y_2(t),\\
          \dot{y}_2(t) &= -(2\pi)^2 y_1(t),\\
          y_1(0) &= 1,\\
          y_2(0) &= 0.
      \end{split}
  \end{equation}
  Simple linear system with exact solution $y_1(t) = \cos(2\pi t)$, $y_2(t) = -2\pi\sin(2\pi t)$.

4. Prothero–Robinson equation
  \begin{equation}
    \begin{split}
      \dot{y}(t) &= \cos(t) - \delta (y(t) - \sin(t)),\\
      y(0) &= 0.
    \end{split}
  \end{equation}
  Linear scalar equation with exact solution $\sin(t)$, stiff for large $\delta$.

5. $\exp$
  \begin{equation}
    \begin{split}
      \dot{y}(t) &= \lambda y(t),\\
      y(0) &= 1.
    \end{split}
  \end{equation}
  Standard test equation with solution $y(t)=\exp(\lambda t)$.

6. Lorenz system

  \begin{equation}
    \begin{split}
      \frac{dx}{dt} &= \sigma (y - x), \\
      \frac{dy}{dt} &= x (\rho - z) - y, \\
      \frac{dz}{dt} &= x y - \beta z,
    \end{split}
  \end{equation}
  with default value of parameters $\sigma = 10$, $\beta = 8/3$, $\rho = 28$.

# A note about implicit methods

For implicit methods (currently Euler) we use Newton iterations. This mean Jacobi matrix is needed. We provide two options.

1. User supplies right-hand sied `F(u, t)` implemented with `jax.numpy` and the derivative is extracted with `jax.jacfwd`.

2. User provides the method-dependent matvec.

  To give an exampe, we consider implicit Euler method. Integration step reads
  \begin{equation}
      u(t_1) - u(t_0) = (t_1 - t_0) F(u(t_1), t_1).
  \end{equation}
  Newton iteration for this nonlinear equation is
  \begin{equation}
      u^{n+1} = u^{n} - \underbrace{\left(I - (t_1 - t_0)\frac{\partial F(u^{n}, t_1)}{\partial u^{n}}\right)^{-1}\left(u^{n+1} - u^{n} - (t_1 - t_0)F(u^{n}, t_1)\right)}_{\text{equation-dependent matvec}}.
  \end{equation}
  So the user provides a function that implements the underlined operation above. This function takes `u, u_F, t, h, s=1`:
  1. `u` $=u^{n+1}$
  2. `u_F` $=\left(u^{n+1} - u^{n} - (t_1 - t_0)F(u^{n}, t_1)\right)$
  3. `t` $=t_1$
  4. `h` $=t_1 - t_0$
  5. `s=1` multiplies each term with $F$, comes from the rescaling of the interval to $[-1, 1]$, correct value is provided internally.

  _Example:_
  For logistic equation we will have
  ```
    exact = lambda x: 1 / (1 + jnp.exp(-x))
    F = lambda u, t: u * (1 - u)
    inv_dF = lambda u, u_F, t, h, s: u_F / (1 - s*h*(1-2*u))
  ```

# Test 1: errors are small, residuals are small

In [4]:
N_points = 100
t0, t1 = 0.0, 1/2
t = (t1 - t0) * (Chebyshev.Chebyshev_grid(N_points) + 1)/2 + t0

integrators = {
    "RK4": RK4.integrator, 
    "Explicit Euler": Explicit_Euler.integrator,
    "Implicit Euler": Implicit_Euler.integrator,
    "Implicit Euler (jac)": Implicit_Euler.integrator_J,
}

for integrator in integrators:
    print(integrator)
    for equation in equations.equations_list:
        equation_data = equations.get_ODE(equation)
        if "exact" in equation_data:
            exact = equation_data["exact"]
            F = equation_data["F"]
            inv_dF = equation_data["inv_dF"]

            exact_solution = exact(t)
            exact_solution = jnp.expand_dims(exact_solution, 0) if exact_solution.ndim == 1 else exact_solution
            u0 = exact_solution[:, 0]

            if integrator == "Implicit Euler (jac)":
                values = integrators[integrator](u0, F, inv_dF, N_points, t0, t1, 1)
            elif integrator == "Implicit Euler":
                values = integrators[integrator](u0, F, N_points, t0, t1, 1)
            else:
                values = integrators[integrator](u0, F, N_points, t0, t1)

            error = jnp.linalg.norm(jnp.ravel(values - exact_solution), ord=jnp.inf)
            res = jnp.linalg.norm(jnp.ravel(utils.residual(values, F, t0, t1)), ord=jnp.inf)

            print(equation)
            print("\terror ", error)
            print("\tresidual ", res)
        
        else:
            pass
    print("\n")

RK4
exp
	error  1.4422907312905409e-11
	residual  1.1282752510055616e-11
Logistic
	error  1.5287771049088406e-13
	residual  1.589769982324185e-13
Harmonic oscillator
	error  5.410830758640792e-07
	residual  1.2887481926071587e-07
Prothero–Robinson
	error  4.4754755457177e-12
	residual  5.4383442193994824e-12


Explicit Euler
exp
	error  0.0025545531251336406
	residual  0.002004970970679254
Logistic
	error  4.518378414930524e-05
	residual  4.6785564135079505e-05
Harmonic oscillator
	error  0.24610502820855729
	residual  0.2875204459200331
Prothero–Robinson
	error  0.0003184044014300502
	residual  0.0003817578003077471


Implicit Euler
exp
	error  0.0025816102581184275
	residual  0.002026405514558738
Logistic
	error  4.42964326373696e-05
	residual  4.585932671420656e-05
Harmonic oscillator
	error  0.22829854113614534
	residual  0.27039555631355405
Prothero–Robinson
	error  0.00031924579366371386
	residual  0.00038292753698532245


Implicit Euler (jac)
exp
	error  0.0025816102581182054
	r

# Test 2: order of convergence

In [15]:
N_points_1, N_points_2 = 10, 100
t0, t1 = 0, 1
t_1 = (t1 - t0) * (Chebyshev.Chebyshev_grid(N_points_1) + 1)/2 + t0
t_2 = (t1 - t0) * (Chebyshev.Chebyshev_grid(N_points_2) + 1)/2 + t0

integrators = {
    "RK4": RK4.integrator, 
    "Explicit Euler": Explicit_Euler.integrator,
    "Implicit Euler": Implicit_Euler.integrator,
    "Implicit Euler (jac)": Implicit_Euler.integrator_J,
}

for integrator in integrators:
    print(integrator)
    for equation in equations.equations_list:
        equation_data = equations.get_ODE(equation)
        if "exact" in equation_data:
            exact = equation_data["exact"]
            F = equation_data["F"]
            inv_dF = equation_data["inv_dF"]

            E = []
            for t, N_points in zip([t_1, t_2], [N_points_1, N_points_2]):
                exact_solution = exact(t)
                exact_solution = jnp.expand_dims(exact_solution, 0) if exact_solution.ndim == 1 else exact_solution
                u0 = exact_solution[:, 0]

                if integrator == "Implicit Euler (jac)":
                    values = integrators[integrator](u0, F, inv_dF, N_points, t0, t1, 1)
                elif integrator == "Implicit Euler":
                    values = integrators[integrator](u0, F, N_points, t0, t1, 1)
                else:
                    values = integrators[integrator](u0, F, N_points, t0, t1)

                error = jnp.linalg.norm(jnp.ravel(values - exact_solution), ord=jnp.inf)
                E.append(error)

            slope = -jnp.log10(E[1]/E[0]) / jnp.log10(N_points_2/N_points_1)
            print(equation, ", ord =", jnp.round(slope.item(), decimals=2))
        
        else:
            pass
    print("\n")

RK4
exp , ord = 4.1
Logistic , ord = 4.15
Harmonic oscillator , ord = 3.98
Prothero–Robinson , ord = 4.16


Explicit Euler
exp , ord = 0.99
Logistic , ord = 1.03
Harmonic oscillator , ord = 1.45
Prothero–Robinson , ord = 1.04


Implicit Euler
exp , ord = 1.09
Logistic , ord = 0.92
Harmonic oscillator , ord = 0.68
Prothero–Robinson , ord = 1.03


Implicit Euler (jac)
exp , ord = 1.09
Logistic , ord = 0.92
Harmonic oscillator , ord = 0.68
Prothero–Robinson , ord = 1.03




Orders are ok.

Interestingly, for harmonic oscillator (symplectic dynamical system) we observe that the sum of estimated orders of explicit and implicit Euler integrators are approximately $2$. Why is that? Perhaps, it is reasonable to consult

Hairer E, Lubich C, Wanner G. Geometric numerical integration illustrated by the Störmer–Verlet method. Acta numerica. 2003 May;12:399-450.